In [2]:
import utils
import feature_extractor
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
from torch.nn.utils.rnn import pad_sequence, pack_padded_sequence, pad_packed_sequence
import pickle

In [5]:
NUM_CLASSES=23
BATCH_SIZE=64
EPOCHS=10
HIDDEN_SIZE=64
VECTOR_SIZE = 200  # Size of word vectors
WINDOW_SIZE = 5  # Context window size
THREADS = 4  # Number of threads to use for training
CUTOFF_FREQ = 1  # Minimum frequency for a word to be included in vocabulary
TRAINING_SIZE = 10000  
TEST_SIZE = 10

In [12]:
complete_data=utils.read_data("../data/fixed_PIZZA_train.json")
# shuflle data
import random
random.shuffle(complete_data)

In [13]:
data = complete_data[:TRAINING_SIZE]
corpus, top, decoupled = utils.get_train_dataset(data)
entites_output_as_number_labels,intents_output_as_number_labels, input_as_tokenized_string=utils.label_complete_input(corpus, decoupled, top)

In [ ]:
reviews_tokens = []
with open('../data/food_review.txt', 'r', encoding='utf-8') as f:
    for line in f:
        reviews_tokens.append(utils.tokenize_string(utils.clean_string(line.strip())))
print("Done1")
count=0
with open('../data/food_review2.txt', 'r', encoding='utf-8') as f:
    for line in f:
        count+=1
        reviews_tokens.append(utils.tokenize_string(utils.clean_string(line.strip())))
        if count==TRAINING_SIZE:
            break
print("Done2")
all_trainig = []
not_all_corpus, _, _ = utils.get_train_dataset(data)
for i in range(len(not_all_corpus)):
    all_trainig.append(utils.tokenize_string(utils.clean_string(not_all_corpus[i]))) 
print("Done3")
emb_model = feature_extractor.train_gensim_fastext_model(all_trainig+reviews_tokens, VECTOR_SIZE)

Done1
Done2
Done3


In [21]:
emb_model.wv.most_similar('thanks')

[('thanksg', 0.9977567195892334),
 ('thanku', 0.9975824356079102),
 ('thankew', 0.9972459077835083),
 ('thankq', 0.9972450137138367),
 ('thankk', 0.9969680309295654),
 ('thanks-giv', 0.9915528893470764),
 ('thank', 0.9787179231643677),
 ('thanksgiv', 0.9616369009017944),
 ('thannk', 0.9485122561454773),
 ("'thank", 0.9378198385238647)]

In [22]:
class LargeDataset(Dataset):
    def __init__(self, data, labels):
        self.data = data 
        self.labels = labels 
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        return self.data[idx], self.labels[idx]

def collate_fn(batch):
    sequences, labels = zip(*batch)
    #I believe we can transform words into embeddings here
    embeddings=[]
    # print(sequences)
    for seq in sequences:
        x=[]
        for token in seq:
            x.append(emb_model.wv[token])
        embeddings.append(x)
    sequences=embeddings
    labels = [torch.tensor(label, dtype=torch.long) for label in labels]
    padded_labels = pad_sequence(labels, batch_first=True, padding_value=-1)
    sequences = [torch.tensor(seq) for seq in sequences]
    padded_sequences = pad_sequence(sequences, batch_first=True)
    lengths = torch.tensor([len(seq) for seq in sequences], dtype=torch.long)
    return padded_sequences, padded_labels, lengths

class LargeWordRNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(LargeWordRNN, self).__init__()
        self.rnn = nn.LSTM(input_size, hidden_size, batch_first=True, bidirectional=True, num_layers=3)
        self.fc = nn.Linear(hidden_size * 2, num_classes)
    
    def forward(self, x, lengths):
        packed_x = pack_padded_sequence(x, lengths.cpu(), batch_first=True, enforce_sorted=False)
        packed_out, _ = self.rnn(packed_x)
        out, _ = pad_packed_sequence(packed_out, batch_first=True)
        out = self.fc(out)
        return out  

In [23]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(torch.cuda.is_available()) 
print(torch.version.cuda)
print(f'Using device: {device}')

False
None
Using device: cpu


In [24]:
labels = entites_output_as_number_labels
dataset = LargeDataset(input_as_tokenized_string, labels)
dataloader = DataLoader(dataset, batch_size=BATCH_SIZE, collate_fn=collate_fn, shuffle=True, num_workers=0)
model = LargeWordRNN(input_size=VECTOR_SIZE, hidden_size=HIDDEN_SIZE, num_classes=NUM_CLASSES).to(device)
criterion = nn.CrossEntropyLoss(ignore_index=-1)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
for epoch in range(EPOCHS): 
    for padded_sequences, padded_labels, lengths in dataloader:
        padded_sequences=padded_sequences.to(device)
        padded_labels=padded_labels.to(device)
        lengths=lengths.to(device)
        optimizer.zero_grad()
        outputs = model(padded_sequences, lengths)
        loss = criterion(outputs.view(-1, NUM_CLASSES), padded_labels.view(-1))
        loss.backward()
        optimizer.step()
    print(f"Epoch {epoch + 1}, Loss: {loss.item():.4f}")

C:\Users\DELL\AppData\Local\Temp\ipykernel_20720\729685153.py:25: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\torch\csrc\utils\tensor_new.cpp:281.)
  sequences = [torch.tensor(seq) for seq in sequences]


Epoch 1, Loss: 0.1759
Epoch 2, Loss: 0.0220
Epoch 3, Loss: 0.0115
Epoch 4, Loss: 0.0030
Epoch 5, Loss: 0.0021
Epoch 6, Loss: 0.0047
Epoch 7, Loss: 0.0082
Epoch 8, Loss: 0.0006
Epoch 9, Loss: 0.0006
Epoch 10, Loss: 0.0005


In [26]:
dev_data=utils.read_data("../data/fixed_PIZZA_dev.json")
dev_corpus, dev_top = utils.get_dev_dataset(dev_data)
gold_dev_labels,dev_as_tokenized_string=utils.label_complete_dev_bert(dev_corpus, dev_top)

class TestLargeDataset(Dataset):
    def __init__(self, data):
        self.data = data 
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        return self.data[idx]

def test_collate_fn(batch):
    sequences = batch
    #I believe we can transform words into embeddings here
    embeddings=[]
    for seq in sequences:
        x=[]
        for token in seq:
            x.append(emb_model.wv[token])
        embeddings.append(x)
    sequences=embeddings
    sequences = [torch.tensor(seq) for seq in sequences]
    padded_sequences = pad_sequence(sequences, batch_first=True)
    lengths = torch.tensor([len(seq) for seq in sequences], dtype=torch.long)
    return padded_sequences, lengths

dev_dataset = TestLargeDataset(dev_as_tokenized_string)
dataloader = DataLoader(dev_dataset, batch_size=1, collate_fn=test_collate_fn, shuffle=False, num_workers=0)
model_output=[]
for padded_sequences, lengths in dataloader:
    labels = []
    padded_sequences=padded_sequences.to(device).float()
    lengths=lengths.to(device).float()
    outputs = model(padded_sequences, lengths)
    entity_to_num = {"I_NUMBER": 0, "I_SIZE": 1, "I_TOPPING": 2, "I_STYLE": 3, "I_DRINKTYPE": 4, "I_CONTAINERTYPE": 5, "I_VOLUME": 6, "I_QUANTITY": 7, "B_NUMBER": 8, "B_SIZE": 9, "B_TOPPING": 10, "B_STYLE": 11, "B_DRINKTYPE": 12, "B_CONTAINERTYPE": 13, "B_VOLUME": 14, "B_QUANTITY": 15, "I_NOT_TOPPING": 16, "B_NOT_TOPPING": 17,"I_NOT_STYLE": 18, "B_NOT_STYLE": 19, "B_NOT_QUANTITY": 20, "I_NOT_QUANTITY": 21, "NONE": 22}
    for i, out in enumerate(outputs[0]):
        num = torch.argmax(out).int().item()
        labels.append(num)
    model_output.append(labels)

confusion_matrix, accuracy=utils.calc_accuracy(dev_corpus,model_output, gold_dev_labels)
print(confusion_matrix)
print(accuracy)

Wrong prediction in 4 th sentence at 5 th token
Wrong prediction in 4 th sentence at 10 th token
Wrong prediction in 4 th sentence at 11 th token
Wrong prediction in 4 th sentence at 12 th token
Sentence: i'll have one pie along with pesto and ham but avoid olives
Pred: [22, 22, 22, 8, 22, 10, 22, 10, 22, 10, 2, 2, 2]
True: [22, 22, 22, 8, 22, 22, 22, 10, 22, 10, 22, 22, 17]
-------------------------------------------------
Wrong prediction in 5 th sentence at 9 th token
Wrong prediction in 5 th sentence at 10 th token
Wrong prediction in 5 th sentence at 13 th token
Sentence: i need to order one large pizza with ham bacon onions and black olives one medium pizza with sausage and onions and six large cokes
Pred: [22, 22, 22, 22, 8, 9, 22, 22, 10, 2, 2, 22, 10, 8, 8, 9, 22, 22, 10, 22, 10, 22, 8, 9, 12]
True: [22, 22, 22, 22, 8, 9, 22, 22, 10, 10, 10, 22, 10, 2, 8, 9, 22, 22, 10, 22, 10, 22, 8, 9, 12]
-------------------------------------------------
Wrong prediction in 6 th sentence at